In [1]:
!pip install -q crewai crewai['tools'] langchain langchain-community langchain-groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.0/235.0 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.0/134.0 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.9/542.9 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 8.6 MB/s eta 0:00

In [2]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 387.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 98.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.8 MB/s eta 0:00:00


In [3]:
!npm install -q localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙
added 22 packages in 3s
⠙
⠙3 packages are looking for funding
⠙  run `npm fund` for details
⠙

In [20]:
%%writefile agents.py
import os
from crewai import Agent, LLM
from langchain_groq import ChatGroq
from search_tools import SearchTools
from calculator_tools import CalculatorTools

# Set your API keys
os.environ["GROQ_API_KEY"] = "YOUR_GROQ_API_KEY"
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"  # Add your OpenAI API key here
os.environ["GEMINI_API_KEY"] = "YOUR_GOOGLE_API_KEY"

# Initialize the Google Gemini model
# llm = ChatGroq(temperature=0.5, groq_api_key=os.getenv("GROQ_API_KEY"), model_name="groq/mixtral-8x7b-32768")

llm = LLM(
    temperature=0.5,
    model='gemini/gemini-1.5-pro-latest',
    api_key=os.getenv('GEMINI_API_KEY')
)

class TripAgents:

  def weather_expert_agents(self):
    return Agent(
        role="Weather Expert",
        goal="Weather expert of city for a range",
        backstory="An expert in analyzing travel condition of a city for a range",
        tools=[
            SearchTools.search_internet
        ],
        verbose=True,
        llm=llm
    )
  def local_expert(self):
    return Agent(
        role="Local Expert at this city",
        goal="Provide the BEST insights about the selected city",
        backstory="""A knowledgeable local guide with extensive information
        about the city, it's attractions and customs""",
        tools=[
            SearchTools.search_internet
        ],
        verbose=True,
        llm=llm
    )

  def travel_concierge(self):
    return Agent(
        role="Amazing Travel Concierge",
        goal="""Create the most amazing travel itineraries with budget and
        packing suggestions for the city""",
        backstory="""Specialist in travel planning and logistics with decades of experience""",
        tools=[
            SearchTools.search_internet,
            CalculatorTools.calculate
        ],
        verbose=True,
        llm=llm
    )


Overwriting agents.py


In [13]:
%%writefile search_tools.py
import json
import os

import requests
from langchain.tools import tool

os.environ['SERPER_API_KEY'] = "YOUR_SERPER_API_KEY"

class SearchTools:

  @tool("Search from internet")
  def search_internet(query):
    """Search the internet for information based on the provided query."""
    top_result_to_return = 3
    url = "https://google.serper.dev/search"
    payload = json.dumps({"q": query})
    headers = {
        'X-API-KEY': os.environ['SERPER_API_KEY'],
        'content-type': 'application/json'
    }

    response = requests.request('POST', url, headers=headers, data=payload)

    if 'organic' not in response.json():
      return "Sorry, I could'nt find anything about that, there could be an error with your serper api key."
    else:
      results = response.json()['organic']
      string = []
      for result in results[:1]:
        try:
          string.append('\n'.join([
              f"Title: {result['title']}", f"Link:{result['link']}",
              f"Snippet: {result['snippet']}", "\n-----------------"
          ]))
        except KeyError:
          next
      return '\n'.join(string)

Overwriting search_tools.py


In [14]:
%%writefile calculator_tools.py
from langchain.tools import tool

class CalculatorTools:
  @tool("Make a calculation")
  def calculate(operation):
    """Perform a mathematical calculation based on the provided operation."""
    try:
      return eval(operation)
    except SyntaxError:
      return "Error: Invalid syntax in mathematical expression"


Overwriting calculator_tools.py


In [15]:
%%writefile tasks.py
from crewai import Task
from textwrap import dedent
from datetime import date

class TripTasks:
  def weather_forecast(self, agent, city, range):
    return Task(
        description = dedent(f"""
        Check for weather forecast in destination city.
        destination city: {city}
        Trip Date: {range}
        """),
        agent = agent,
        expected_output = "Detailed weather report on the chosen city"
    )

  def trip_info(self, agent, city, range):
    return Task(
        description = dedent(f"""
        Identify to 5 tourist attractions of the destination city
        on that trip date range.

        Trip Date: {range}
        destination city: {city}
        """),
        agent = agent,
        expected_output = "Comprehensive city guide of the city"
    )

  def plan_task(self, agent, origin, city, range):
    return Task(
        description = dedent(f"""
            Expand this guide into a full 7-day travel itinerary, packing suggestions,
            and a budget breakdown

            Trip Date: {range},
            Traveling from: {origin}
            destination city: {city}
          """),
        agent = agent,
        expected_output = "Complete expanded travel plan with daily schedule and budget breakdown"
    )


Overwriting tasks.py


In [16]:
%%writefile main.py
import streamlit as st
from crewai import Crew
from textwrap import dedent
from agents import TripAgents
from tasks import TripTasks

class TripCrew:
  def __init__(self, origin, city, date_range):
    self.city = city
    self.origin = origin
    self.date_range = date_range

  def run(self):
    agents = TripAgents()
    tasks = TripTasks()

    weather_forecast_agent = agents.weather_expert_agents()
    local_expert_agent = agents.local_expert()
    travel_concierge_agent = agents.travel_concierge()

    weather_tasks = tasks.weather_forecast(
        weather_forecast_agent,
        self.city,
        self.date_range
    )

    trip_info_tasks = tasks.trip_info(
        local_expert_agent,
        self.city,
        self.date_range
    )

    plan_task = tasks.plan_task(
        travel_concierge_agent,
        self.origin,
        self.city,
        self.date_range
    )

    crew = Crew(
        agents = [weather_forecast_agent, local_expert_agent, travel_concierge_agent],
        tasks = [weather_tasks, trip_info_tasks, plan_task],
        verbose = True
    )

    result = crew.kickoff()
    return result

def trip_planner_app():
  st.title("Trip Planner Crew")
  st.write("## Welcome to this Streamlit-based Trip Planner!")

  location = st.text_input("From where", placeholder="origin")
  city = st.text_input("Go to", placeholder="destination")
  date_range = st.text_input("Date range of the trip", placeholder="date range")

  if st.button("Plan My Trip!"):
    if location and city and date_range:
      trip_crew = TripCrew(location, city, date_range)
      st.write("Running the trip planner...")

      result = trip_crew.run()

      st.write("### Here is your trip plan")
      print(result)
      st.write(result)
    else:
      st.warning("Please fill in all fields to plan your trip")

if __name__ == "__main__":
    trip_planner_app()

Overwriting main.py


In [21]:
!streamlit run main.py &>/content/logs.txt &

In [18]:
!wget -q -O - ipv4.icanhazip.com

34.41.48.234


In [22]:
!npx localtunnel --port 8501

⠙your url is: https://ten-bobcats-stare.loca.lt
^C
